In [ ]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

In [ ]:
# input parameters for the simualtions

input_parameters={
    "minE": 10**15, #eV, 
    "maxE": 10**18, #eV,    
    "gamma":  -2.5, # slope of the spectrum
    #relation between energy and s38
    "A": 10**12, #eV
    "B": 1.2,
    #attenuation true numbers
    "alpha":0.5,
    "beta": -3.0,
    "maxTheta": 45 #degrees
}
## set the sid to generate exactly the same samples
np.random.seed(10)  
print(input_parameters)
#number of events to generate
events = 10000

import os
filepath='/home/ioana/src/icecube/Noemi/stage/cic_att' 
#filepath='/home/noemie/stage/cic_att' 
os.chdir(filepath)
%run data_functions.py

from data_functions import generate_toy_data

data = generate_toy_data(events, input_parameters)


In [ ]:
###attenuation curve
from data_functions import obtain_attenuation
nr_of_bins = 10
intensity = 500
samples = 50
import time
start_time = time.time()

fit_results, fitted_data = obtain_attenuation(data, nr_of_bins, intensity, samples)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
import math
groups = fitted_data
sample = fit_results
cos_ref = np.cos(math.radians(25))**2
cos2 = np.linspace(0.5, 1, 20)-cos_ref

a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(sample, [16, 50, 84],
                                      axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s_ref = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

# Plot a subset of the samples
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")
for a, b, f in sample[np.random.randint(len(sample), size=400)]:
    plt.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="b", alpha=0.03)
    plt.plot(cos2, s38_mcmc[0] * (b_mcmc[0] * cos2**2 + a_mcmc[0] * cos2 + 1), color="b", lw=0.5, alpha=0.8)

plt.plot(cos2, f * (input_parameters['beta'] * cos2**2 + input_parameters['alpha'] * cos2 + 1), color="r", 
         alpha=1)
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")

In [ ]:
import corner

true_vals = [input_parameters['alpha'], input_parameters['beta'], None]  
fig = corner.corner(sample, truths=true_vals, truth_color='r',
                    labels=["$a$","$b$", "$s_{ref}$"], quantiles=[0.16, 0.84],
                    show_titles=True, title_kwargs={"fontsize": 20}, color='b')
#fig.savefig("")
a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(sample, [16, 50, 84],
                                                axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s38 = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

In [ ]:
fig2, ax2 =plt.subplots(figsize=(10,6))
temp, groups = set_intensity(data, 10)
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")
plt.ylim(100, 1000)
plt.legend()


In [ ]:
%run data_functions.py
from data_functions import set_intensity
data['s38'] = data.s125/(b_mcmc[0] * (data.cos2-cos_ref)**2 + a_mcmc[0] * (data.cos2-cos_ref) + 1)

print(data)
temp, groups = set_intensity(data, 4, 's38')
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")

plt.legend()